In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

import pdblp
import numpy as np
import pandas as pd

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.data.fields import fields_reference
from api.data.processing import post_process
from api.data.base import tables
import utils

In [ ]:
timestamp = f"{pd.Timestamp.today():%Y%m%d_%H%M%S}"

In [ ]:
root_path = Path('/dev/projects/credit/backend/')
data_path = root_path / 'data'

### Bloomberg Connection

In [ ]:
bq = BlpQuery().start()

In [ ]:
searches = [
    'at1',
    'lt2',
    'sp',
    'snp'
]

In [ ]:
tickers = []
for search in searches:
    tickers.append(bq.bsrch(f"FI:{search}"))

In [ ]:
tickers = pd.concat(tickers, keys=searches, sort=False).droplevel(1).rename('security')

In [ ]:
tickers.to_csv(data_path / 'temp' / f"searches_{timestamp}.csv" )

In [ ]:
res = bq.bdp(
    securities=tickers.to_list(),
    fields=[*fields],
)

In [ ]:
res.to_csv(data_path / 'temp' / f"bonds_reference_{timestamp}.csv" , index=False)

In [ ]:
res = post_process(res, tickers)

In [ ]:
res.to_csv(data_path / 'bonds_reference.csv', index=False)

### Bugfix

#### 1. Add new fields

In [ ]:
# res = pd.read_csv(data_path / 'temp' / 'bonds_reference_20220525_193116.csv')
# tickers = pd.read_csv(data_path / 'temp' / 'searches_20220525_193116.csv', index_col=0).squeeze()
res = pd.read_csv(data_path / 'bonds_reference.csv')

In [ ]:
res_ = bq.bdp(
    securities=res['security'].to_list(),
    fields=['ISSUER_EQUITY_TICKER'],
).rename(fields_reference, axis=1)

In [16]:
for field in fields_reference:
    fn = field.get('fn')
    if fn:
        res_[field['field']] = fn(res_[field['field']])
        

In [20]:
res_ = res_.rename({field['field']: field['name'] for field in fields_reference}, axis=1)

In [22]:
res = res.merge(res_.set_index('security'), left_on='security', right_index=True, how='left')

#### 2. Add new Instruments

In [ ]:
new_tickers = tickers.loc[~tickers.isin(res['security'])]

In [ ]:
res_ = bq.bdp(
    securities=new_tickers.to_list(),
    fields=[*fields],
).rename(fields, axis=1)

In [ ]:
res_ = post_process(res_, new_tickers)

In [ ]:
res = pd.concat([res, res_])

In [24]:
res.to_csv(data_path / 'bonds_reference.csv', index=False)